In [51]:
import os
import re
import pymorphy2
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 300)
import gensim
import matplotlib.pyplot as plt

from langdetect import detect
from gensim.models import KeyedVectors
from nltk import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy.spatial.distance import cosine
from nltk.corpus import stopwords

%matplotlib inline
float_formatter = lambda x: "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

In [2]:
import logging
logger = logging.getLogger("imported_module")

## Gensim library

In [3]:
logger.setLevel(logging.CRITICAL)
en_vectors_file = 'numberbatch-en.txt'
uk_vectors_file = 'news.lowercased.tokenized.word2vec.300d'

uk_vectors = KeyedVectors.load_word2vec_format(uk_vectors_file, binary=False)

ik_vectors = uk_vectors.get_vector('слово')
uk_vectors.most_similar('слово')

/olhasempienv/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('дієслово', 0.6502863764762878),
 ('слівце', 0.6484909653663635),
 ('словосполучення', 0.6456568241119385),
 ('гасло', 0.5913079977035522),
 ('слово**', 0.555127739906311),
 ("прислів'я", 0.5407627820968628),
 ('письмо', 0.5235773324966431),
 ('прізвище', 0.52119380235672),
 ('пророцтво', 0.5125285983085632),
 ('ремесло', 0.5058826804161072)]

In [4]:
king_vectors = uk_vectors.get_vector('король')
man_vectors = uk_vectors.get_vector('чоловік')
woman_vectors = uk_vectors.get_vector('жінка')

uk_vectors.most_similar(positive=['жінка', 'король'], negative=['чоловік'])
# or
uk_vectors.similar_by_vector(king_vectors - man_vectors + woman_vectors)

[('королева', 0.6083585023880005),
 ('дама', 0.5521167516708374),
 ('принцеса', 0.5491373538970947),
 ('знаменитість', 0.544516921043396),
 ('жінка', 0.5441809892654419),
 ('танцівниця', 0.5311182737350464),
 ('панянка', 0.5286626815795898),
 ('кінозірка', 0.5274341106414795),
 ('актриса', 0.52556973695755),
 ('матуся', 0.5253670811653137)]

## Read Data

In [5]:
def get_files(directory_path):
    '''Read all files in the given directory to a dataframe'''
    data = []

    for filename in os.listdir(directory_path):
        with open(os.path.join(directory_path, filename)) as f:
            docs = re.split(r'\n{2,4}', f.read())
            # docs = f.read().split('\n\n')
            for doc in docs:
                if doc.strip('\n'):
                    data.append((filename[:len(filename)-4], doc))

    return pd.DataFrame(data, columns=['category', 'text'])

df_data = get_files('1551')
df_data.head(5)

,category,text
0,Укладання-та-ремонт-тротуарної-плитки--брущатки-,"2837901\nНа тротуарному покритті відсутні частини цегли (шт. з 8-10). Необхідно ”залатати” цю діру. Раніше в квітні 2014 залишав скаргу (на фото), але жодних змін не відбулося. Писали, що Київенерго вело роботи й має все за собою прибрати. Але на набережній Київенерго нічого не робило. прошу лік..."
1,Укладання-та-ремонт-тротуарної-плитки--брущатки-,3127477\nНезадовільний стан тротуарної плитки. Пропоную звернутися до балансоутримувача з пропозицією відновити належний стан.
2,Укладання-та-ремонт-тротуарної-плитки--брущатки-,"2692310\nЯ, як мешканець Оболонського району занепокоєний ситуацією, яка склалася з якістю пішохідної доріжки біля будинку по вул. Героїв Дніпра 36б. Вимагаю від Вас терміново звернути увагу на цю проблему та вирішити її (ВІДНОВИТИ ЗАДОВІЛЬНИЙ СТАН ПІШОХІДНОЇ ДОРІЖКИ, ВСТАНОВИТИ НА МІСЦЕ ЗСУНУТУ..."
3,Укладання-та-ремонт-тротуарної-плитки--брущатки-,"2691881\nЯ, як мешканець Оболонського району занепокоєний ситуацією, яка склалася з якістю пішохідної доріжки біля під’їзду №2 будинку за адресою вул. Героїв Дніпра 36а. Вимагаю від Вас терміново звернути увагу на цю проблему та вирішити її (ВІДНОВИТИ ЗАДОВІЛЬНИЙ СТАН ПІШОХІДНОЇ ДОРІЖКИ, ЗАМІНИТ..."
4,Укладання-та-ремонт-тротуарної-плитки--брущатки-,"2690960\nЯ, як мешканець Оболонського району занепокоєний ситуацією, яка склалася з якістю пішохідної доріжки біля будинку по вул. Героїв Дніпра 36а. Вимагаю від Вас терміново звернути увагу на цю проблему та вирішити її (ВІДНОВИТИ ЗАДОВІЛЬНИЙ СТАН ПІШОХІДНОЇ ДОРІЖКИ, ЗАМІНИТИ ПОШКОДЖЕНІ ПЛИТИ, ..."


## Document embeddings

In [62]:
morph = pymorphy2.MorphAnalyzer(lang='uk')
stop_words = stopwords.words('english')

def get_doc_emb(text):
    tokens = [morph.parse(i.lower())[0].normal_form for i in word_tokenize(text)
              if morph.parse(i.lower())]  # and not i.lower() in stop_words
    arr_tokens = np.array([uk_vectors.get_vector(i) for i in tokens if i in uk_vectors.vocab])
    if arr_tokens.size != 0:
        max_ar = arr_tokens.max(axis=0)
        avg_ar =  arr_tokens.sum(axis=0) / arr_tokens.sum(axis=0).sum()
        arr_doc = np.concatenate((max_ar,avg_ar))
        return arr_doc.reshape(1, -1)
    else:
        return 'Empty'

def get_similar_doc(doc2vec, docs, vec_docs):
    vec_sim = np.array([cosine_similarity(doc2vec, doc) for doc in vec_docs])
    return docs[vec_sim.argmax()]

In [50]:
doc = df['doc2vec'].tolist()[1]
docs = df['text'][2:].tolist()
vec_docs =  df['doc2vec'][2:].tolist()

print("Similar documents:\n")
print(df['text'].tolist()[1], '\n')
print(get_similar_doc(doc, docs, vec_docs))

Similar documents:

3320914
Скарга! Протягом двох років!!! мешканці будинку №8 по вул. Чавдар  подають колективні скарги  в  ЖЕК - ТОВ Фірму Компас. Мета цих скарг- це примусити ЖЕК полагодити- укласти бруківку на придомовій території (біля будинку №8 по вул. Чавдар). Але скарги-прохання не діють! на ЖЕК -ТОВ Фірма Компас і дана організація не виконує своїх прямих обов’язків , а саме - не виконує ремонт-укладку бруківки біля будинку.
Просимо здійснити Всі можливі методи впливу на цю організацію. 
Розібрана бруківка створює небезпеку для мешканців, особливо дітей, які можуть травмуватися.
Прохання розібратися і примусити укласти бруківку до 10 вересня 2015 року. 

2585899
Доводжу до Вашого відома, що станом на 05.10.2013р. в квартирі відсутнє опалення. Опитування мешканців дому виявило, що обігрів ведеться в усіх квартирах, окрім тих, що знаходяться на одному стояку з нашою. До того, 03.10.2013р. одна з мешканок вже подала заяву до ЖЕД №222, її запевнили, що 04.10.2013р. прийде фахівець

Interestingly, what languages were detected

In [68]:
df_whole = pd.read_pickle('proc_data.pkl')
# df_whole.loc[:,'lang'] = df_whole['text'].map(detect_lang)
df_whole['lang'].value_counts()

uk             87094
bg              1678
mk               356
Not Defined      135
de                15
hu                13
en                 8
cs                 7
pl                 3
et                 2
ca                 2
sl                 2
nl                 1
cy                 1
id                 1
tl                 1
ro                 1
sk                 1
Name: lang, dtype: int64

## Preprocessed Data

In [63]:
def detect_lang(text):
    try:
        lang = detect(text)
    except:
        lang = 'Not Defined'
    return lang

def prepocess_data(df_data):
    '''
    Filter categories with more than 1000 and less than 2000 documents per category,
    filter only documents only in Ukrainian
    and add new columns doc2vec
    '''
    df_data.drop_duplicates(subset ="text", keep = False, inplace = True)
    
    df_filter = df_data.groupby('category').count()
    filter_columns = df_filter[(df_filter['text'] >= 1000) & (df_filter['text'] <= 2000)].index.values
    df = df_data[df_data['category'].map(lambda x: True if x in filter_columns else False)]
     
    # only ukrainian
    df.loc[:,'lang'] = df['text'].map(detect_lang)
    df = df[df['lang'] == 'uk']

    df.loc[:, 'doc2vec'] = df['text'].map(get_doc_emb)
    df = df[df['doc2vec'] != 'Empty']
    df = df[df['doc2vec'].map(lambda x: False if np.any(np.isnan(x)) else True)]
    df.loc[:, 'doc2vec_reshape'] = df['doc2vec'].map(lambda x: x[0])

    return df

df = prepocess_data(df_data)
df.head(5)

/olhasempienv/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/olhasempienv/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/olhasempienv/lib/python3.6/site-packages/pandas/core/ops.py:1122: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x, y, op)


,category,text,lang,doc2vec,doc2vec_reshape
1387,Ремонт-дахів,"2677863\nУ жилому приміщенні під час дощу з даху тече вода,стеля вже вкрилась грибком,в приміщенні живуть діти.Жек-на колективне звернення 10 місяців тому відповів що будівля обстежена і по можливості буде усунено затікання на протязі року .звертались ще не раз --роботи не ведуться.ДОПОМОЖІТЬ!!!...",uk,"[[3.015301, 3.636102, 2.199778, 2.775461, 3.197708, 2.703665, 3.789194, 2.860864, 4.130527, 3.492456, 2.113725, 1.669964, 3.366665, 2.212062, 2.477931, 4.118515, 3.353093, 3.116153, 2.160366, 3.252919, 3.138227, 3.367545, 4.811172, 1.814569, 2.299579, 2.113835, 3.876257, 3.632734, 2.980686, 2.03...","[3.015301, 3.636102, 2.199778, 2.775461, 3.197708, 2.703665, 3.789194, 2.860864, 4.130527, 3.492456, 2.113725, 1.669964, 3.366665, 2.212062, 2.477931, 4.118515, 3.353093, 3.116153, 2.160366, 3.252919, 3.138227, 3.367545, 4.811172, 1.814569, 2.299579, 2.113835, 3.876257, 3.632734, 2.980686, 2.036..."
1388,Ремонт-дахів,"2665757\nУ жилому приміщенні під час дощу з даху тече вода,стеля вже вкрилась грибком,в приміщенні живуть діти.Жек-на колективне звернення 10 місяців тому відповів що будівля обстежена і по можливості буде усунено затікання на протязі року .звертались ще не раз --роботи не ведуться.ДОПОМОЖІТЬ!!!...",uk,"[[3.015301, 3.636102, 2.199778, 2.775461, 3.197708, 2.703665, 3.789194, 2.860864, 4.130527, 3.492456, 2.113725, 1.669964, 3.366665, 2.212062, 2.477931, 4.118515, 3.353093, 3.116153, 2.160366, 3.252919, 3.138227, 3.367545, 4.811172, 1.814569, 2.299579, 2.113835, 3.876257, 3.632734, 2.980686, 2.03...","[3.015301, 3.636102, 2.199778, 2.775461, 3.197708, 2.703665, 3.789194, 2.860864, 4.130527, 3.492456, 2.113725, 1.669964, 3.366665, 2.212062, 2.477931, 4.118515, 3.353093, 3.116153, 2.160366, 3.252919, 3.138227, 3.367545, 4.811172, 1.814569, 2.299579, 2.113835, 3.876257, 3.632734, 2.980686, 2.036..."
1390,Ремонт-дахів,"2652058\nУ жилому приміщенні під час дощу з даху тече вода,стеля вже вкрилась грибком,в приміщенні живуть діти.Жек-на колективне звернення 10 місяців тому відповів що будівля обстежена і по можливості буде усунено затікання на протязі року .звертались ще не раз --роботи не ведуться.ДОПОМОЖІТЬ!!!...",uk,"[[2.182084, 3.636102, 2.199778, 2.775461, 3.197708, 2.703665, 3.789194, 2.860864, 4.130527, 3.492456, 2.113725, 1.669964, 3.366665, 2.212062, 2.477931, 4.118515, 3.353093, 3.116153, 2.160366, 3.252919, 3.138227, 3.367545, 4.811172, 1.814569, 2.299579, 2.113835, 3.876257, 3.632734, 2.980686, 2.03...","[2.182084, 3.636102, 2.199778, 2.775461, 3.197708, 2.703665, 3.789194, 2.860864, 4.130527, 3.492456, 2.113725, 1.669964, 3.366665, 2.212062, 2.477931, 4.118515, 3.353093, 3.116153, 2.160366, 3.252919, 3.138227, 3.367545, 4.811172, 1.814569, 2.299579, 2.113835, 3.876257, 3.632734, 2.980686, 2.036..."
1391,Ремонт-дахів,"3382871\nКоли будуть проводити ремонт покрівлі, нам сказали, що вже і в план наш будинок включений і все готово, а ремонт не починається??? Скоро підуть дощі, чи ніхто робити не буде, а це лише чергові обіцянки перед виборами???",uk,"[[1.374014, 3.260054, 2.163689, 2.445364, 2.335736, 5.842228, 3.612487, 4.848104, 2.471985, 2.356168, 3.85318, 1.386743, 3.287878, 1.841844, 5.137277, 4.118515, 3.2305, 3.116153, 2.436169, 4.372425, 2.014224, 0.783148, 2.848231, 2.971451, 2.585541, 2.48073, 2.153415, 5.108227, 3.357788, 3.208922...","[1.374014, 3.260054, 2.163689, 2.445364, 2.335736, 5.842228, 3.612487, 4.848104, 2.471985, 2.356168, 3.85318, 1.386743, 3.287878, 1.841844, 5.137277, 4.118515, 3.2305, 3.116153, 2.436169, 4.372425, 2.014224, 0.783148, 2.848231, 2.971451, 2.585541, 2.48073, 2.153415, 5.108227, 3.357788, 3.208922,..."
1393,Ремонт-дахів,3367399\nПрошу терміново (до сезону дощів) відремонтувати димохід на даху будинка (фото додані) для запобігання залиття його дощем та застпання снігом,uk,"[[1.222078, 3.023618, 2.171771, 3.172965, 2.974267, 2.565827, 2.28465, 2.32813, 2.014141, 3.492456, 2.555812, 0.799856, 2.808588, 1.755683, 2.9263

In [54]:
# df.to_pickle('proc_data_short.pkl')
# df = pd.read_pickle('proc_data_short.pkl')

In [64]:
df.groupby('category').count()

,text,lang,doc2vec,doc2vec_reshape
category,,,,
Інші-технічні-недоліки-стану-ліфту,866,866,866,866
Будівництво-дооблаштування-дитячого-майданчику,744,744,744,744
Встановлення-та-експлуатація-лічильників-на-водопостачання,689,689,689,689
Відсутнє-ХВП,806,806,806,806
Відсутність-освітлення-на-опорних-стовпах-за-відсутності-несправності-лампочок,585,585,585,585
ГЛ--Несанкціонована-торгівля-,1013,1013,1013,1013
Незадовільна-температура-опалення,713,713,713,713
Освітлення-в-приміщенні-й-при-вході-в-нього,784,784,784,784
Перевірка-наявності-дозволів-на-виконання-будівельних-робіт,854,854,854,854


In [58]:
df['lang'].value_counts()

uk    10731
Name: lang, dtype: int64

## Train Test Split

In [65]:
X = df['doc2vec_reshape'].tolist()

y = df['category'].tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

## Standard KNN Classifier (bigger dataset)

In [60]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train) 

y_pred = neigh.predict(X_test)

print(classification_report(y_true=y_test, y_pred=y_pred))

                                                                                  precision    recall  f1-score   support

                                              Інші-технічні-недоліки-стану-ліфту       0.15      0.38      0.22       260
                                  Будівництво-дооблаштування-дитячого-майданчику       0.27      0.51      0.35       254
                      Встановлення-та-експлуатація-лічильників-на-водопостачання       0.32      0.40      0.36       215
                                                                    Відсутнє-ХВП       0.31      0.49      0.38       278
  Відсутність-освітлення-на-опорних-стовпах-за-відсутності-несправності-лампочок       0.16      0.23      0.19       177
                                                   ГЛ--Несанкціонована-торгівля-       0.58      0.47      0.52       336
                                               Незадовільна-температура-опалення       0.33      0.30      0.31       238
                       

## KNN Classifier with cosine metric (bigger dataset)

In [66]:
neigh = KNeighborsClassifier(n_neighbors=3, metric=cosine)
neigh.fit(X_train, y_train) 

y_pred = neigh.predict(X_test)

print(classification_report(y_true=y_test, y_pred=y_pred))

                                                                                  precision    recall  f1-score   support

                                              Інші-технічні-недоліки-стану-ліфту       0.16      0.36      0.22       281
                                  Будівництво-дооблаштування-дитячого-майданчику       0.26      0.48      0.33       253
                      Встановлення-та-експлуатація-лічильників-на-водопостачання       0.30      0.53      0.38       219
                                                                    Відсутнє-ХВП       0.35      0.50      0.41       277
  Відсутність-освітлення-на-опорних-стовпах-за-відсутності-несправності-лампочок       0.18      0.22      0.20       179
                                                   ГЛ--Несанкціонована-торгівля-       0.59      0.49      0.54       312
                                               Незадовільна-температура-опалення       0.35      0.31      0.33       250
                       

## Conclusions
### Small dataset

Improvements with dataset provided on practice:

1. Document embeddings are calculated as a concatenated vector of two vectors. First one is retrieved from word embeddings as maximum vector and the second one as an average vector. The document embedding vector now is 600-dimensional. The results are slightly better:<br>
f1-score micro avg: 0.54 -> 0.54<br>
f1-score macro avg: 0.50 -> 0.53

2. Adding lemmatization from pymorphy2<br>
f1-score micro avg: 0.54 -> 0.58<br>
f1-score macro avg: 0.53 -> 0.55

3. Adding language detection<br>
f1-score micro avg: 0.58 -> 0.60<br>
f1-score macro avg: 0.55 -> 0.58

### Bigger dataset
Next steps were performed on a bigger dataset, but I trimmed it to 10'731 documents because it takes too long to process the whole dataset.

4. Next step of imporving the model is to use cosine metric.<br>
The model performs slightly better.<br>
f1-score micro avg: 0.32 -> 0.33<br>
f1-score macro avg: 0.30 -> 0.32<br>

5. Do not include stop words makes no improvements.<br>
f1-score micro avg: 0.33 -> 0.33<br>
f1-score macro avg: 0.32 -> 0.31<br>

### Whole dataset
I run Standard KNN on the whole dataset. The results are not good:<br>
f1-score micro avg: 0.21<br>
f1-score macro avg: 0.13

## Class 6 - Neural Network

Embed - ReLU - Softmax

Adam optim

In [ ]:
classes = dict(enumerate(set(y_train)))
classes

# X_train

import torch
import torch.nn as nn
from torch.autograd import Variable

# Defining Model
class FeedForwardNN(nn.Module):
    def __init__(self, inp_size, require_bias=True):
        super(FeedForwardNN, self).__init__()
        self.relu = nn.ReLU(in_features=inp_size, out_features=1, bias=require_bias)
        
#         self.inp_size = inp_size
        
    def forward(self, inp_batch):
        
        out = self.relu(inp_batch)
        
        # Softmax layer
        tag_scores = F.log_softmax(out, dim=1)
        
        return tag_scores

# GPU
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
# model = MyRNN().to(device)
torch.cuda.is_available()

# # Creating sample data for sum of 3 elements
# X = []
# for i in range(2):
#     for j in range(2):
#         for k in range(2):
#             X.append([i, j, k])

# y = list(map(lambda x: x[0] + x[1] + x[2], X))
# X = Variable(torch.FloatTensor(X), requires_grad=False)
# y = Variable(torch.FloatTensor(y), requires_grad=False)

# ?
X = Variable(torch.FloatTensor(X_train), requires_grad=False)
y = Variable(torch.FloatTensor(y_train), requires_grad=False)

# Creating model and defining loss function and optimizer
model = FeedForwardNN()
loss_fn = nn.MSELoss()
optim = torch.optim.Adam(model.parameters())
losses_visualize = []

# Training the model
for epoch in range(600):
    optim.zero_grad()
    
    out = model(X)
    loss = loss_fn(out, y)
    losses_visualize.append(loss.item())
    
    loss.backward()
    optim.step()

plt.plot(losses_visualize)